In [1]:
import numpy as np
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
import time
import pandas as pd

import sys
sys.path.append("/data01/Tomo2Mesh/scratchpad/polymer_am_eaton/code")
from rw_utils import read_raw_data_1X
from void_mapping import void_map_gpu
from params import pixel_size_1X as pixel_size
from tomo2mesh.misc.num_density import num_density
from params import save_path
from tomo2mesh.structures.voids import VoidLayers

In [9]:
b = 4
num_layers = 5
sample_tag = 2
z_max = []
voids_list = []
voids_all = VoidLayers()
rdf = pd.read_csv(save_path)

y_pos_list = []
for layer in range(1,1+num_layers):
    info = rdf[(rdf["sample_num"] == int(sample_tag)) & (rdf["layer"] == int(layer))].iloc[0]
    y_pos_list.append(info['sample_y'])

y_pos_list = np.asarray(y_pos_list)*1.0e3/(pixel_size*b)
y_pos_list = y_pos_list - y_pos_list[0]
z_max = np.uint32(np.diff(y_pos_list))


In [10]:
# adjust cents, cpts with y_pos
# clip z values to last 50 (lower clip), first 50 (upper clip)

In [11]:
for i in range(num_layers):
    layer = i+1
    projs, theta, center, dark, flat = read_raw_data_1X(sample_tag, layer)
    #voids.select_by_size(200, pixel_size_um = pixel_size) # remove later
    voids = void_map_gpu(projs, theta, center, dark, flat, b, pixel_size)
#     if layer == 6 or layer == 12:
#         voids.select_z_slab(0,286)
#     else:
#         voids.select_z_slab(0,50)
    voids.select_z_slab(0,50)
    voids.calc_max_feret_dm()
    surf = voids.export_void_mesh_with_texture("sizes")
    surf.write_ply(f'/data01/Eaton_Polymer_AM/ply_files/sample{sample_tag}_layer{layer}_top_slabs.ply')
    print(layer)
    print("added a layer")
    

/data01/Tomo2Mesh/scratchpad/polymer_am_eaton/code/recon.py:91: FutureWarning: cupy.fft.cache.clear_plan_cache is experimental. The interface can change in the future.
  cp.fft.config.clear_plan_cache()


OutOfMemoryError: Out of memory allocating 4,230,144,000 bytes (allocated so far: 0 bytes).

In [ ]:
for i in range(num_layers):
    layer = i + 1
    projs, theta, center, dark, flat = read_raw_data_1X(sample_tag, layer)
    #voids.select_by_size(200, pixel_size_um = pixel_size) # remove later
    voids = void_map_gpu(projs, theta, center, dark, flat, b, pixel_size)
#     if layer == 6 or layer == 12:
#         voids.select_z_slab(0,286)
#     else:
#         voids.select_z_slab(236,286)
    voids.select_z_slab(236,286)
    voids.calc_max_feret_dm()
    surf = voids.export_void_mesh_with_texture("sizes")
    surf.write_ply(f'/data01/Eaton_Polymer_AM/ply_files/sample{sample_tag}_layer{layer}_bottom_slabs.ply')
    print(layer)
    print("added a layer")
    